In [15]:
import pandas as pd
import re
import string

#change dataset path
df = pd.read_csv('./data/processed/ukrainian_tweets_processed.csv')

/tmp/ipykernel_23254/1518327901.py:6: DtypeWarning: Columns (0,1,6,7,8,10,17,18,20,21,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/processed/ukrainian_tweets_processed.csv')


In [22]:
df = df[~df['text_processed'].isna()]

In [23]:
preprocessed_documents = df['text_processed'].tolist()


In [24]:
# !pip install cleantext

In [25]:
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk.tokenize import word_tokenize
# from cleantext import clean

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()

# # Function to preprocess documents
# def preprocess(document):
#     # Tokenize
#     words = word_tokenize(document.lower())
#     # Remove stopwords and punctuations
#     filtered_words = [word for word in words if word.isalnum() and not word in stop_words]
#     # Lemmatize
#     lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
#     return ' '.join(lemmatized_words)

# # Preprocess all documents
# preprocessed_documents = [preprocess(doc) for doc in documents]


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from gensim import matutils

tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=4)
tfidf = tfidf_vectorizer.fit_transform(preprocessed_documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()


In [33]:
model = NMF(n_components=10, init='random', random_state=0)
W = model.fit_transform(tfidf)
H = model.components_

In [34]:
# Extract the top words for each topic
n_top_words = 10
topics = []
for topic_idx, topic in enumerate(H):
    top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
    top_features = [tfidf_feature_names[i] for i in top_features_ind]
    topics.append(top_features)


In [35]:
# # Convert the list of top words into a list of lists of words
# texts = [[word for word in doc.lower().split() if word in tfidf_feature_names] for doc in preprocessed_documents]

# # Create a Gensim dictionary
# dictionary = Dictionary(texts)

# # Convert the dictionary and the corpus
# corpus = [dictionary.doc2bow(text) for text in texts]

# # Calculate the coherence score using Gensim
# coherence_model = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_v')
# coherence_score = coherence_model.get_coherence()

# print('Coherence Score:', coherence_score)

In [36]:
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from multiprocessing import Pool

# Convert tfidf_feature_names to a set for faster lookup
tfidf_feature_names_set = set(tfidf_feature_names)

def preprocess_document(doc):
    # Lower and split the document only once, and filter using the set
    return [word for word in doc.lower().split() if word in tfidf_feature_names_set]

# Use parallel processing to optimize the conversion of documents into lists of words
with Pool() as pool:
    texts = pool.map(preprocess_document, preprocessed_documents)




In [37]:
# Create a Gensim dictionary
dictionary = Dictionary(texts)

# Convert the dictionary and the corpus
corpus = [dictionary.doc2bow(text) for text in texts]

# Calculate the coherence score using Gensim
coherence_model = CoherenceModel(topics=topics, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

print('Coherence Score:', coherence_score)

Coherence Score: 0.5320522264948864


In [38]:
topics

[['україна',
  'зсу',
  'росія',
  'украин',
  'россиїй',
  'войнасукраиной',
  'агрессияроссии',
  'вторжениероссии',
  'війна',
  'війнапутін'],
 ['слава',
  'україна',
  'герой',
  'воїн',
  'перемогти',
  'зсу',
  'смерть',
  'славаукраїна',
  'гера',
  'ворог'],
 ['ебати',
  'іти',
  'путін',
  'нахувати',
  'путина',
  'росія',
  'хуйло',
  'росіянин',
  'війнути',
  'померти'],
 ['так',
  'робити',
  'але',
  'виглядати',
  'зробити',
  'єс',
  'працювати',
  'жити',
  'казати',
  'питання'],
 ['анімація',
  'отже',
  'війна',
  'украин',
  'война',
  'орієнтовно',
  'противник',
  'скласти',
  'течія',
  'врятований'],
 ['дякувати',
  'паня',
  'підтримка',
  'ви',
  'зсу',
  'пані',
  'приємно',
  'захисник',
  'навзаєм',
  'великий'],
 ['__', 'то', 'ви', 'бути', 'та', 'але', 'пані', 'мен', 'ні', 'ти'],
 ['день', 'шо', 'знати', 'та', 'не', 'хотіти', 'робити', 'як', 'мати', 'ви'],
 ['це',
  'точно',
  'правда',
  'скарб',
  'життя',
  'місто',
  'жах',
  'зробити',
  'смішно',


In [ ]:
df['extractedts'].apply(lambda x : x.split(' ')[0]).value_counts()

extractedts
2022-08-19    17663
2022-08-20     5516
Name: count, dtype: int64